In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from __future__ import division
import os
import re
import sys
import random
import time
import binascii
from bisect import bisect_right
from heapq import heappop, heappush
import pandas as pd
import numpy as np
from tabulate import tabulate
from itertools import combinations
#from beautifultable import BeautifulTable


# This is the number of components in the resulting MinHash signatures.
numHashes = 100


# It ships with data set sizes 100, 1000, 2500, and 10000.
numDocs = 100
trainFile = "/content/drive/MyDrive/Colab Notebooks/data/articles_" + str(numDocs) + ".train.txt"
testFile = "/content/drive/MyDrive/Colab Notebooks/data/articles_" + str(numDocs) + ".test.txt"

with open('/content/drive/MyDrive/Colab Notebooks/data/file3.txt', 'w', encoding="utf-8") as dataFile:
  
    # Iterate through list
    for names in [trainFile, testFile]:
  
        # Open each file in read mode
        with open(names, encoding="utf-8") as infile:
  
            # read the data from file1 and file2 and write it in file3
            dataFile.write(infile.read())
        dataFile.write("\n")


#truthFile = "./data/articles_" + str(numDocs) + ".truth.txt"

#Convert Documents To Sets of Shingles

print ("Shingling articles are listed below :- ")

# The current shingle ID value to assign to the next new shingle we encounter.
curShingleID = 0

# Create a dictionary of the articles, mapping the article identifier (e.g.,"t8470") to the list 
#of shingle IDs that appear in the document.
docsAsShingleSets = {};
  

f = open('/content/drive/MyDrive/Colab Notebooks/data/file3.txt', "r" , encoding="utf8")

docNames = []

t0 = time.time()

totalShingles = 0

docNM = []

for i in range(0, numDocs):
  
  words = f.readline().split(" ")  
  
  docID = words[0]
  
  # Maintain a list of all document IDs.  
  docNames.append(docID)
    
  del words[0]  
  
  shinglesInDoc = set()
  
  # For each word in the document...
  for index in range(0, len(words) - 2):

    shingle = words[index] + " " + words[index + 1] + " " + words[index + 2]
    #print(shingle)
    #print("\n")
    

    # Hash the shingle to a 32-bit integer.
    crc = binascii.crc32(shingle.encode('utf8')) & 0xffffffff
    
    # Add the hash value to the list of shingles for the current document.  
    shinglesInDoc.add(crc)
  
  docsAsShingleSets[docID] = shinglesInDoc
  #print (shinglesInDoc)
  #print("\n")
  
  # Count the number of shingles across all documents.
  totalShingles = totalShingles + (len(words) - 2)

f.close()  



# Report how long shingling took.
print ("\nShingling " + str(numDocs) + " docs took %.2f sec." % (time.time() - t0))
 
print ('\nAverage shingles per doc: %.2f' % (totalShingles / numDocs))

# =============================================================================
#                     Define Triangle Matrices
# =============================================================================

# Calculate the number of elements needed in our triangle matrix
numElems = int(numDocs * (numDocs - 1) / 2)

# Initialize two empty lists to store the similarity values
#for the actual Jaccard Similarity values
JSim = [0 for x in range(numElems)]
#for the estimated Jaccard Similarities found by comparing the MinHash signatures.
estJSim = [0 for x in range(numElems)]



# Define a function to map a 2D matrix coordinate into a 1D index.
def getTriangleIndex(i, j):
  # If i == j that's an error.
  if i == j:
    sys.stderr.write("Can't access triangle matrix with i == j")
    sys.exit(1)
  # If j < i just swap the values.
  if j < i:
    temp = i
    i = j
    j = temp
  
  # Calculate the index within the triangular array.
  # This fancy indexing scheme is taken from pg. 211 of:
  # http://infolab.stanford.edu/~ullman/mmds/ch6.pdf
  # But I adapted it for a 0-based index.
  # Note: The division by two should not truncate, it
  #       needs to be a float. 
  k = int(i * (numDocs - (i + 1) / 2.0) + j - i) - 1
  
  return k


# =============================================================================
#                 Calculate Jaccard Similarities
# =============================================================================
# This is included here to show how much slower it is than
# the MinHash approach.

# Calculating the Jaccard similarities gets really slow for large numbers
# of documents.

if numDocs <= 2500:
    print ("\nCalculating Jaccard Similarities...")
    #t0 = time.time()

    # For every document pair...
    for i in range(0, numDocs):
      
      # Print progress every 100 documents.
      if (i % 100) == 0:
        print ("  (" + str(i) + " / " + str(numDocs) + ")")

      # Retrieve the set of shingles for document i.
      s1 = docsAsShingleSets[docNames[i]] #docNames = ID from data
      
      for j in range(i + 1, numDocs):
        # Retrieve the set of shingles for document j.
        s2 = docsAsShingleSets[docNames[j]]
        
        # Calculate and store the actual Jaccard similarity.
        JSim[getTriangleIndex(i, j)] = (len(s1.intersection(s2)) / len(s1.union(s2)))
        # Retrieve the estimated similarity value for this pair.

    # Calculate the elapsed time (in seconds)
    elapsed = (time.time() - t0)
    print ("\nCalculating all Jaccard Similarities took %.2fsec" % elapsed)

# Delete the Jaccard Similarities, since it's a pretty big matrix.    
del JSim


Shingling articles are listed below :- 

Shingling 100 docs took 0.04 sec.

Average shingles per doc: 243.69

Calculating Jaccard Similarities...
  (0 / 100)

Calculating all Jaccard Similarities took 0.18sec
